In [81]:
from sunpy.net import hek
import pandas as pd 
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from sunpy.time import parse_time
import numpy as np
client = hek.HEKClient()

In [2]:
tstart = '2017-09-01'
tend = '2020-10-01'

In [4]:
t1 = time.time()
result = client.search(hek.attrs.Time(tstart, tend),
                       hek.attrs.EventType('FL'),
                       hek.attrs.OBS.Observatory == 'GOES')
t2 = time.time()
print(t2 - t1)

41.83053398132324


In [7]:
new_table = result[['event_starttime', 'event_peaktime',
                       'event_endtime', 'fl_goescls', 'ar_noaanum', ]]

In [26]:
new_table['peak_time_hour'] = [x.hour for x in pd.to_datetime(new_table['event_peaktime'])]

In [31]:
new_table['date'] = [x[0:10] for x in new_table['event_peaktime']]

In [32]:
daytime_flares = new_table[(new_table['peak_time_hour']>8) & (new_table['peak_time_hour']<19)]

In [40]:
daytime_flares = daytime_flares.to_pandas()

In [41]:
daytime_flares['date'].unique()

array(['2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
       '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
       '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
       '2017-09-20', '2017-09-24', '2017-09-25', '2017-09-26',
       '2017-09-27', '2017-09-28', '2017-09-29', '2017-09-30',
       '2017-10-04', '2017-10-05', '2017-10-06', '2017-10-08',
       '2017-10-20', '2017-10-22', '2017-11-13', '2017-11-14',
       '2017-11-16', '2017-11-18', '2017-11-23', '2017-11-27',
       '2017-12-01', '2017-12-10', '2017-12-20', '2017-12-22',
       '2018-01-05', '2018-01-12', '2018-01-18', '2018-01-20',
       '2018-01-28', '2018-01-30', '2018-02-05', '2018-02-06',
       '2018-02-07', '2018-02-08', '2018-02-09', '2018-02-10',
       '2018-02-11', '2018-02-12', '2018-02-14', '2018-02-17',
       '2018-02-26', '2018-02-28', '2018-03-02', '2018-03-29',
       '2018-03-30', '2018-03-31', '2018-04-01', '2018-04-03',
       '2018-04-15', '2018-04-18', '2018-04-20', '2018-

In [43]:
baseurl = "https://data.lofar.ie/2017/09/10/bst/kbt/rcu357_1beam/"

In [61]:
date = parse_time(daytime_flares['event_peaktime'][0])

In [76]:
def find_lofar_files(date):
    """
    Find I-LOFAR dynamic spectra for date given.
    
    Parameters
    ----------
    date : ~`datetime.datetime` or ~`astropy.Time`
    
    Returns
    -------
    `list` of available files
    """
    baseurl = "https://data.lofar.ie/{:s}/bst/kbt/rcu357_1beam/".format(date.strftime("%Y/%m/%d"))
    file_urls = []
    try:
        opn = urlopen(baseurl)
        try:
            soup = BeautifulSoup(opn, "html.parser")
            for link in soup.find_all("a"):
                href = link.get("href")
                if href.endswith("00X.dat"):# or href.endswith("00Y.dat"):
                    file_urls.append(baseurl + href)
        except:
            print("No file of interest")
    except:
        print("No page")
    return file_urls

In [77]:
urls = []
for t in daytime_flares['date'].unique():
    date = parse_time(t)
    print(t)
    files = find_lofar_files(date)
    if len(files)>0:
        urls.append(files[0])
    else:
        urls.append(None)

2017-09-01
2017-09-02
2017-09-03
2017-09-04
2017-09-05
No page
2017-09-06
No page
2017-09-07
2017-09-08
2017-09-09
2017-09-10
2017-09-11
No page
2017-09-12
No page
2017-09-20
No page
2017-09-24
2017-09-25
No page
2017-09-26
No page
2017-09-27
No page
2017-09-28
No page
2017-09-29
2017-09-30
No page
2017-10-04
No page
2017-10-05
No page
2017-10-06
No page
2017-10-08
2017-10-20
No page
2017-10-22
No page
2017-11-13
No page
2017-11-14
No page
2017-11-16
No page
2017-11-18
No page
2017-11-23
No page
2017-11-27
No page
2017-12-01
2017-12-10
2017-12-20
No page
2017-12-22
2018-01-05
2018-01-12
No page
2018-01-18
No page
2018-01-20
2018-01-28
2018-01-30
No page
2018-02-05
No page
2018-02-06
No page
2018-02-07
No page
2018-02-08
No page
2018-02-09
No page
2018-02-10
2018-02-11
2018-02-12
No page
2018-02-14
No page
2018-02-17
No page
2018-02-26
No page
2018-02-28
No page
2018-03-02
2018-03-29
No page
2018-03-30
2018-03-31
2018-04-01
2018-04-03
2018-04-15
2018-04-18
No page
2018-04-20
2018-04-21


In [82]:
urls= np.array(urls)

In [85]:
len(urls[urls!=None])

40

In [88]:
for u in urls[0:10]:
    print(u)

https://data.lofar.ie/2017/09/01/bst/kbt/rcu357_1beam//2017/09/01/bst/kbt/rcu357_1beam/20170901_105216_bst_00X.dat
https://data.lofar.ie/2017/09/02/bst/kbt/rcu357_1beam//2017/09/02/bst/kbt/rcu357_1beam/20170902_103626_bst_00X.dat
https://data.lofar.ie/2017/09/03/bst/kbt/rcu357_1beam//2017/09/03/bst/kbt/rcu357_1beam/20170903_084439_bst_00X.dat
https://data.lofar.ie/2017/09/04/bst/kbt/rcu357_1beam//2017/09/04/bst/kbt/rcu357_1beam/20170904_065647_bst_00X.dat
None
None
https://data.lofar.ie/2017/09/07/bst/kbt/rcu357_1beam//2017/09/07/bst/kbt/rcu357_1beam/20170907_104152_bst_00X.dat
https://data.lofar.ie/2017/09/08/bst/kbt/rcu357_1beam//2017/09/08/bst/kbt/rcu357_1beam/20170908_083815_bst_00X.dat
https://data.lofar.ie/2017/09/09/bst/kbt/rcu357_1beam//2017/09/09/bst/kbt/rcu357_1beam/20170909_075303_bst_00X.dat
https://data.lofar.ie/2017/09/10/bst/kbt/rcu357_1beam//2017/09/10/bst/kbt/rcu357_1beam/20170910_070804_bst_00X.dat


In [95]:
parse_time(daytime_flares['date']).strftime('%Y/%m/%d')[0] in urls

False

In [100]:
lofar_data = []
for i in range(len(daytime_flares)):
    if any(parse_time(daytime_flares['date']).strftime('%Y/%m/%d')[i] in s for s in urls[urls!=None]):
        lofar_data.append('yes')
        print('yes')
    else:
        lofar_data.append('nay')
        print('no')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
no
no
no
no
no
no
no
no
no
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
yes
yes
no
no
no
no
no
no
no
no
yes
yes
no
no
no
no
no
no
no
no
no
yes
yes
no
yes
yes
no
no
yes
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
no
no
no
no
no
no
no
no
no
yes
no
yes
yes
yes
yes
yes
yes
yes
yes
no
no
no
yes
yes
yes
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
yes
yes
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
yes
no
no
yes
yes
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
yes
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no

In [101]:
daytime_flares['lofar_data'] = lofar_data

In [107]:
lofar_events = daytime_flares[daytime_flares['lofar_data']=='yes']

In [109]:
lofar_events.reset_index(drop=True, inplace=True)

In [110]:
lofar_events

,event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum,peak_time_hour,date,lofar_data
0,2017-09-01T09:37:00,2017-09-01T09:43:00,2017-09-01T09:52:00,B7.1,12674,9,2017-09-01,yes
1,2017-09-01T10:25:00,2017-09-01T10:28:00,2017-09-01T10:30:00,B3.7,12674,10,2017-09-01,yes
2,2017-09-01T15:04:00,2017-09-01T15:07:00,2017-09-01T15:09:00,B4.8,12674,15,2017-09-01,yes
3,2017-09-01T15:10:00,2017-09-01T15:13:00,2017-09-01T15:15:00,B5.1,12674,15,2017-09-01,yes
4,2017-09-01T15:31:00,2017-09-01T15:37:00,2017-09-01T15:41:00,C2.0,12674,15,2017-09-01,yes
...,...,...,...,...,...,...,...,...
86,2019-01-29T10:21:00,2019-01-29T10:34:00,2019-01-29T10:40:00,C1.9,12733,10,2019-01-29,yes
87,2019-01-29T15:03:00,2019-01-29T15:22:00,2019-01-29T15:36:00,B3.5,12733,15,2019-01-29,yes
88,2019-02-18T16:00:00,2019-02-18T16:04:00,2019-02-18T16:05:00,A1.4,0,16,2019-02-18,yes
89,2019-05-15T16:42:00,2019-05-15T16:56:00,2019-05-15T17:03:00,B2.0,12741,16,2019-05-15,yes
